In [47]:
import numpy as np
import scipy.io as sio
from PIL import Image
from numpy import linalg as LA
import matplotlib.pyplot as plt

# load data

mat_contents = sio.loadmat("mnist_digits.mat") # mat_contents['X'].shape = (10000, 784)
dataset = np.concatenate((mat_contents['X'], mat_contents['Y']), axis=1) # X, Y = dataset[:,:-1], dataset[:,-1] 

# split data

def split(dataset, ratio):
    msk = np.random.rand(len(dataset)) < ratio
    return dataset[msk], dataset[~msk]

train_data, test_data = split(dataset, 0.7)

# build percepton

def kernel(x, y, p=1):
    return (1 + np.dot(x, y)) ** p

def percepton_ker(train_data, test_datapoint, max_iter, digit, p=1):
        
    n_samples, n_features = train_data.shape
    if len(train_data)==0: return [[]]

    alpha = np.zeros(n_samples)
    K = np.zeros((n_samples, n_samples))
    y = list(map(lambda x: 1 if x else -1, train_data[:,-1] == digit))
    
    # Compute Kernel Matrix
    for i in range(n_samples):
        for j in range(n_samples):
            K[i,j] = kernel(train_data[i], train_data[j], p)
            
    for t in range(max_iter):
        for i in range(n_samples):
            if np.sign( np.sum(K[:,i]*alpha*y) ) * y[i] <= 0:
                alpha[i] += 1

    # Compute Test Kernel
    K_test = np.zeros((1, n_samples))
    for i in range(n_samples):
        K_test[:,i] = kernel(train_data[i], test_datapoint)
        
    return np.sign( np.sum(K_test*alpha*y) )
        

def percepton_accuracy_ker(train_data, test_data, max_iter, digit, p):
    
    count = 0
    total = len(test_data)
    if total==0:
        return 0
        
    for datapoint in test_data:
        
        res = percepton_ker(train_data, datapoint, max_iter, digit, p)
        if (res == 1 and datapoint[-1] == digit) \
        or (res == -1 and datapoint[-1] != digit):
            count += 1
            
    return count/total

In [ ]:
percepton_ker(train_data, test_data[0], 1, 0, 1)

In [ ]:
max_iter = 1
digit = 0
p = 5
percepton_accuracy_ker(train_data, test_data, max_iter, digit, p)